[![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=grey&color=yellow&logoColor=white&label=%20&style=flat-square)](https://colab.research.google.com/github/ap-mdi-it/ml-courses/blob/main/book/ml_principles/labs/imgrec_taco_train.ipynb)

Navigeer naar deze notebook op GitHub: [https://github.com/ap-mdi-it/ml-courses/blob/main/book/ml_principles/labs/imgrec_taco_train](https://github.com/ap-mdi-it/ml-courses/blob/main/book/ml_principles/labs/imgrec_taco_train)  
  
Via bovenstaande link kan je deze notebook openen in Google Colaboratory. In die omgeving kunnen we gebruik maken van gratis quota voor GPUs (en TPUs). GPU acceleratie is hier sterk aanbevolen voor zowel model training als model inference.

# Colab setup
- Als je de notebook voor het eerst opent op Colab, kies je in het menu rechts bovenaan `Connect > Change runtime type`: `Python 3` en `T4 GPU`.
- Pas nadat de GPU Runtime is opgestart ga je verder met onderstaande installatie van het [ultralitics](https://github.com/ultralytics/ultralytics) package.
- ⚠️ Bij deze waarschuwing `Warning: This notebook was not authored by Google` selecteer je `Run anyway`.

In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
from pprint import pprint

import kagglehub
import yaml
from ultralytics import YOLO


In [ ]:
# Enable TensorBoard logging
!yolo settings datasets_dir=/content/taco
!yolo settings tensorboard=True

⚠️ Om bovenstaande aanpassingen in werking te stellen moet je nu de notebook runtime herstarten

# Dataset

In [ ]:
# Download the dataset
path = kagglehub.dataset_download("vencerlanz09/taco-dataset-yolo-format")

print(path)

In [ ]:
# Copy the contents of the directory instead of moving it to avoid root permission issues
destination = "./taco"
if not os.path.exists(destination):
    shutil.copytree(path, destination)
path = destination
print(path)

In [ ]:
# Update the meta data (see taco#1.ipynb)
with open(path + "/data.yaml") as f:
    meta = yaml.safe_load(f)

# Update image paths
meta["train"] = "./train/images"
meta["val"] = "./valid/images"
meta["test"] = "./test/images"

# Swap Bottle cap and Bottle labels (see)
bottle_cap_idx = meta["names"].index("Bottle cap")
bottle_idx = meta["names"].index("Bottle")
meta["names"][bottle_cap_idx], meta["names"][bottle_idx] = (
    meta["names"][bottle_idx],
    meta["names"][bottle_cap_idx],
)

# write back to file
with open(path + "/data.yaml", "w") as f:
    yaml.dump(meta, f)

pprint(meta)

# Training (hyper) Parameters

In [ ]:
# Define training parameters
epochs = 30
batch_size = 32
imgsz = 640  # Image size
optimizer_type = "AdamW"  # AdamW optimizer (recommended for better regularization)
lr = 1e-4
weight_decay = 1e-4

# Model Training

In [ ]:
# Initialize YOLOv11 model (pre-trained weights)
model = YOLO("yolov8s.pt")

# check the architecture
# print(model)

In [ ]:
# Initialize YOLOv11 model (pre-trained weights)
model = YOLO("yolov8s.pt")

Start Tensorboard voor live monitoring.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs

In [ ]:
# Train model with Cosine Annealing learning rate scheduler
model.train(
    data=path + "/data.yaml",
    epochs=epochs,
    batch=batch_size,
    imgsz=imgsz,
    optimizer=optimizer_type,
    lr0=lr,  # Initial learning rate
    weight_decay=weight_decay,
    save=True,  # Save the best model
    save_period=1,  # Save model every 10 epochs
    # val=True,  # Evaluate on validation set
)